In [12]:
self = afm_scanner_logic
import logic.gwyfile as gwy

In [73]:
def _save_esr_to_gwyddion(self, dataobj,filename, datakeys=None,prefix=None):
        """
            save_esr_to_gwyddion(): writes esr data object to gwy container file
            input:  
            - dataobj: proteusQ data object of from dataobj['data_key']
            - filename: file path to save object
            - prefix:  name to be prefixed to all head objects
            
            requirements:
            dataobj must contain keys {coord0[], coord1[], coord2[],
                                        data[,,], data_std[,,], data_fit[,,], parameters[]}
        """

        # helper function for color generation
        # r/g/bspec = (min,max,number,startvalue)
        def colors(rspec=(0,1,10,0), gspec=(0,1,10,0), bspec=(0,1,10,0)):
            reds = np.linspace(*rspec[:3])
            reds =np.concatenate([reds[np.argwhere(reds >= rspec[-1])], 
                                reds[np.argwhere(reds < rspec[-1])]]).flatten()

            greens = np.linspace(*gspec[:3])
            greens =np.concatenate([greens[np.argwhere(greens >= gspec[-1])], 
                                greens[np.argwhere(greens < gspec[-1])]]).flatten()

            blues = np.linspace(*bspec[:3])
            blues =np.concatenate([blues[np.argwhere(blues >= bspec[-1])], 
                                blues[np.argwhere(blues < bspec[-1])]]).flatten()

            while True:
                for r in reds:
                    for g in greens:
                        for b in blues:
                            yield [('color.red', r), 
                                   ('color.green', g), 
                                   ('color.blue',b)]
        

        # check for existance of valid object names
        if datakeys is None:
            datakeys = list(dataobj.keys())
        else:
            if isinstance(datakeys,str):
                datakeys = list(datakeys)
            alloweddatakeys = list(dataobj.keys())
            for n in datakeys:
                if not n in alloweddatakeys:
                    self.log.error("_save_esr_to_gwyddion():Invalid object name specified '{n}'")

        # determine if anything is to be done
        if np.sum(dataobj['data']) == 0.0:
            return False

        # Output 
        # overall object container
        esrobj = gwy.objects.GwyContainer()

        # ESR mean data (quite dense)
        # create curves
        xdata = dataobj['coord2_arr']  # microwave frequency
        curves = []
        getcolors = colors((0,1,5,0.9),(0,1,5,0),(0,0.8,5,0))   # point color generator

        #  specifies 1st curve is points(1), 2nd curve is line(2), others hidden(0)
        ltypes = [1,2] 
        for j in range(dataobj['coord1_arr'].shape[0]):
            for i in range(dataobj['coord0_arr'].shape[0]):
                cols = next(getcolors)
    
                # measured data
                ydata = dataobj['data'][j,i,:]
                curve = gwy.objects.GwyGraphCurveModel(xdata=xdata, ydata=ydata)
                curve.update(cols)
                curve['description'] = f"coord[{j},{i}]"
                curve['type'] = ltypes.pop(0) if ltypes else 0
                curve['line_style'] = 0 
                curves.append(curve)

                # fit data
                ydata = dataobj['data_fit'][j,i,:]
                curve = gwy.objects.GwyGraphCurveModel(xdata=xdata, ydata=ydata)
                curve.update(cols)
                curve['description'] = f"coord[{j},{i}]_fit"
                curve['type'] = ltypes.pop(0) if ltypes else 0
                curve['line_style'] = 0 
                curves.append(curve)

        esrgraph = gwy.objects.GwyGraphModel()
        esrgraph['title'] = 'ESR: data per pixel'
        esrgraph['curves'] = curves
        esrgraph['x_unit'] = gwy.objects.GwySIUnit(unitstr='Hz')
        esrgraph['y_unit'] = gwy.objects.GwySIUnit(unitstr='c/s')
        esrgraph['bottom_label'] = 'Microwave Frequency'
        esrgraph['left_label'] = dataobj['nice_name'] + ' Count'

        esrobj['/0/graph/graph/1'] = esrgraph 
        esrobj['/0/graph/graph/1/visible'] = False 

        esrobj.tofile(filename) 
        return esrobj

In [123]:
with open('G:\\Data\\Qudi_Data\\2023\\03\\20230325\\AttoDRY2200_Pi3_SPM\\esr_array_raw.pickle', 'rb') as f:
    esr_data = pickle.load(f)
datobj = esr_data['esr_fw']
filename = 'G:\\Data\\Qudi_Data\\2023\\03\\20230331\\AttoDRY2200_Pi3_SPM\\20230331-1309-01_FST_resave_data_esr_gwy_test.gwy'

In [124]:
esrobj = _save_esr_to_gwyddion(self, datobj,filename, datakeys=None,prefix=None)

In [125]:
print(esrobj.keys())
print(esrobj['/0/graph/graph/1'].keys())

odict_keys(['/0/graph/graph/1', '/0/graph/graph/1/visible'])
odict_keys(['title', 'curves', 'x_unit', 'y_unit', 'bottom_label', 'left_label'])


In [156]:
objout = gwy.objects.GwyContainer()

In [157]:
datobj.keys()

dict_keys(['data', 'data_std', 'data_fit', 'coord0_arr', 'coord1_arr', 'coord2_arr', 'measured_units', 'scale_fac', 'si_units', 'nice_name', 'params', 'display_range'])

In [158]:
meas = datobj

In [159]:
coord0 = meas['coord0_arr']
coord1 = meas['coord1_arr']
coord2 = meas['coord2_arr']
data_si = meas['data'] #* scalefactor
xy_units = gwy.objects.GwySIUnit(unitstr='m')
z_units = gwy.objects.GwySIUnit(unitstr=meas['si_units'])
measname = 'esr_fw' + ":" + meas['nice_name']

In [160]:
x,y,z = (data_si.shape[0], data_si.shape[1], data_si.shape[2])
new_data = np.zeros((z,x,y))
for i in range(z):
    new_data[i] = data_si[:,:,i]
data_si = new_data

In [161]:
img = gwy.objects.GwyBrick(data=data_si.flatten(),
                 xres=len(datobj['coord0_arr']), yres=len(datobj['coord1_arr']), zres=len(datobj['coord2_arr']),
                 xreal=None, yreal=None, zreal=None,
                 xoff=None, yoff=None, zoff=None,
                 si_unit_x=xy_units, si_unit_y=xy_units, 
                 si_unit_z=gwy.objects.GwySIUnit(unitstr='Hz'), si_unit_w=z_units,
                 typecodes=None)
# img.data = data_si

xstart = datobj['coord0_arr'][0]
xstop = datobj['coord0_arr'][-1]
img.xoff = xstart
img.xreal = xstop - xstart

ystart = datobj['coord1_arr'][0]
ystop = datobj['coord1_arr'][-1]
img.yoff = ystart
img.yreal = ystop - ystart


zstart = datobj['coord2_arr'][0]
zstop = datobj['coord2_arr'][-1]
img.zoff = zstart
img.zreal = zstop - zstart


In [162]:
basekey = '/' + 'brick/' + '0'
objout[basekey] = img
objout[basekey + '/title'] = measname
objout[basekey + '/visible'] = True


if objout:
    objout.tofile(filename) 

In [169]:
afm_scanner_logic._qafm_scan_array['counts_fw']['params']

{'Parameters for': 'QAFM measurement', 'axis name for coord0': 'X', 'axis name for coord1': 'Y', 'measurement plane': 'XY', 'coord0_start (m)': 5.465e-06, 'coord0_stop (m)': 9.465e-06, 'coord0_num (#)': 100, 'coord1_start (m)': 6.398e-06, 'coord1_stop (m)': 1.0398e-05, 'coord1_num (#)': 100, 'ESR Frequency start (Hz)': 2850000000.0, 'ESR Frequency stop (Hz)': 2902000000.0, 'ESR Frequency points (#)': 27, 'ESR Count Frequency (Hz)': 100.0, 'ESR MW power (dBm)': -5.0, 'ESR Measurement runs (#)': 12, 'Expect one resonance dip': False, 'Optimize Period (s)': None, 'AFM integration time per pixel (s)': 0.004, 'AFM scan speed (s)': 1e-07, 'Measurement parameter list': "['counts', 'b_field', 'Height(Dac)']", 'Measurement start': '2023-03-24T14:36:13.861887', 'Measurement stop': '2023-03-25T02:50:12.547771', 'Total measurement time (s)': 44038.685884}

False